In this notebook we will cover some general features of the [astropy](http://astropy.org) packages.

In [ ]:
%pylab inline

# FITS files

This section uses parts from the [astropy documentation on FITS files](http://docs.astropy.org/en/stable/io/fits/index.html).

We are able to read fits files, manipulate the header, and save it all again. Note that this library was previously called `pyfits`.

In [ ]:
from astropy.io import fits
# from astropy.io import fits as pyfits  # for people not really converted yet from pyfits.
hdulist = fits.open('input.fits')
hdulist.info()

### Working with the data

This is CRIRES data of a star. The data is saved on four CHIPS, and in each CHIP there is 9 x 1D data arrays. Let's take a closer look at the `dtype` of these arrays.

In [ ]:
cols = hdulist[1].columns
print cols

From the CRIRES webpage, I know for plotting purposes we should use the `Extracted_OPT` and the `Wavelength` arrays.

In [ ]:
plot(hdulist[1].data['Wavelength'], hdulist[1].data['Extracted_OPT'])

If we want to plot all of them, we could do something like

In [ ]:
for spectra in hdulist[1:]:  # We don't want the first one
    w, f = spectra.data['Wavelength'], spectra.data['Extracted_OPT']
    plot(w, f / np.median(f))  # Bring the spectra to the same level (rude normalize).

### Working with the header

In [ ]:
hdr = hdulist[1].header
print repr(hdr[0:10]), '\n'  # No need to see it all here...
print hdr['NAXIS2']
print hdr[4]
print hdr.keys()[0:10]  # Easy to loop over the header

In [ ]:
print len(hdr.keys())
hdr['PCOUNT'] = 3.1415  # Updating a header value. Previously use set, but it is depricated.
print repr(hdr[0:10])

### Saving data

As so many other things, this can be done in many ways. Here is one that never fails.

In [ ]:
import numpy as np
x = np.linspace(0, 2*np.pi, 100)
# Generate some fake noisy data
y = np.sin(x) + np.random.random(len(x)) * 0.1

# Generate a header (also what splot@IRAF and ARES uses)
hdr = fits.Header()
print repr(hdr)
hdr['NAXIS1'] = len(x)
hdr['CRVAL1'] = x[0]
hdr['CDELT1'] = x[1] - x[0]
print repr(hdr)

# Save the data
fits.writeto('newfits.fits', y, hdr, clobber=True)

The last parameter `clobber=True` overwrites existing files. Default this is set to `False`.

## Modeling and fitting

In this section we follow the example on [astropy for modeling](http://docs.astropy.org/en/stable/modeling/index.html).

In [ ]:
from astropy.modeling import models, fitting

### Creating a 1D Gaussian model

In [ ]:
g = models.Gaussian1D(amplitude=1.2, mean=0.9, stddev=0.5)

In [ ]:
print g

We can also acces the values in another way (as attributes)

In [ ]:
print g.amplitude
print g.mean
print g.stddev

We can even change them

In [ ]:
g.amplitude = 0.75

Let's try to evaluate the model in some points

In [ ]:
print g(0.1)
print g(3)
print g(np.linspace(0, 2, 10))

In [ ]:
x = np.linspace(-2, 2, 1000) + g.mean
plot(x, g(x))

### Simple 1D model fitting

In [ ]:
# Generate some fake data
np.random.seed(0)  # Because there is no such thing as real random in CS

x = np.linspace(-5., 5., 200)
y = 3 * np.exp(-0.5*(x-1.3)**2 / (0.8**2))  # Creating a gaussian
y += np.random.normal(0, 0.2, x.shape)  # Adding some noise to our data

# Fit the data using a box (trapezoid) model
t_init = models.Trapezoid1D(amplitude=1, x_0=0, width=1, slope=0.5)
fit_t = fitting.LevMarLSQFitter()
t = fit_t(t_init, x, y)

# Fit the data using a Gaussian
g_init = models.Gaussian1D(amplitude=1, mean=0, stddev=1)
fit_g = fitting.LevMarLSQFitter()
g = fit_g(g_init, x, y)

## Plot the data together with the two best fits
plt.plot(x, y, 'k.')

plt.plot(x, t(x), '-b', lw=2, label='Trapezoid')
plt.plot(x, t_init(x), '--r', alpha=0.6) # initial guess

plt.plot(x, g(x), '-r', lw=2, label='Gaussian')
plt.plot(x, g_init(x), '--b', alpha=0.6) # initial guess

plt.xlabel('Position')
plt.ylabel('CCF')

plt.legend(loc=2, frameon=False)